In [1]:
!pip install numpy torch scipy tqdm matplotlib qutip


Defaulting to user installation because normal site-packages is not writeable


In [2]:
from src.data import DensityMatrixDataset

dataset = DensityMatrixDataset(10)
x, y = dataset[0]

print("Input:", x)
print("Input shape:", x.shape)

print("Target shape:", y.shape)


Input: tensor([-0.2717,  0.2302,  0.0579])
Input shape: torch.Size([3])
Target shape: torch.Size([16])


C:\Users\Aryan Soni\PaAC Assingment 2\src\data.py:60: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  self.inputs = torch.tensor(self.inputs, dtype=torch.float32)


In [3]:
import torch
from src.data import DensityMatrixDataset
from src.model import DensityNet
from src.metrics import quantum_fidelity, trace_distance, inference_latency

# Load model
model = DensityNet()
model.load_state_dict(torch.load("outputs/model.pt"))
model.eval()

# Load test sample
dataset = DensityMatrixDataset(100)
x, y = dataset[0]

# True density matrix
L_true = y.view(4,4)
rho_true = L_true @ L_true.T
rho_true = rho_true / torch.trace(rho_true)

# Predicted density matrix
with torch.no_grad():
    L_pred = model(x.unsqueeze(0)).view(4,4)
rho_pred = L_pred @ L_pred.T
rho_pred = rho_pred / torch.trace(rho_pred)

# Metrics
fid = quantum_fidelity(rho_true, rho_pred)
td  = trace_distance(rho_true, rho_pred)
lat = inference_latency(model, x.unsqueeze(0))

print("Fidelity:", fid)
print("Trace Distance:", td)
print("Inference Latency (seconds):", lat)


Fidelity: 0.09116470068693161
Trace Distance: 0.19174811244010925
Inference Latency (seconds): 7.756233215332031e-05


In [4]:
import torch
from src.data import DensityMatrixDataset
from src.model import DensityNet
from src.metrics import quantum_fidelity, trace_distance

# Load model
model = DensityNet()
model.load_state_dict(torch.load("outputs/model.pt"))
model.eval()

dataset = DensityMatrixDataset(200)

fidelities = []
trace_distances = []

with torch.no_grad():
    for i in range(len(dataset)):
        x, y = dataset[i]

        # true rho
        L_true = y.view(4,4)
        rho_true = L_true @ L_true.T
        rho_true = rho_true / torch.trace(rho_true)

        # predicted rho
        L_pred = model(x.unsqueeze(0)).view(4,4)
        rho_pred = L_pred @ L_pred.T
        rho_pred = rho_pred / torch.trace(rho_pred)

        fidelities.append(quantum_fidelity(rho_true, rho_pred))
        trace_distances.append(trace_distance(rho_true, rho_pred))

print("Mean Fidelity:", sum(fidelities)/len(fidelities))
print("Mean Trace Distance:", sum(trace_distances)/len(trace_distances))


Mean Fidelity: 0.09547568844631314
Mean Trace Distance: 0.33955957256257535
